In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
for i in range(1,12):   
    base_url = f'https://www.iranjib.ir/jax/showarchive.php?p=1&id={i}'
    r = requests.get(base_url)
    r.status_code
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find('table', {'class':'cellpadding5'})
    data = []
    rows = soup.find_all('tr')
    for row in rows:
        cells = row.find_all('td')
        if len(cells) == 3:
            link_tag = cells[0].find('a')
            title = link_tag.text.strip()
            link = link_tag['href']
            date_time = cells[2].text.strip()

            data.append({
                'title': title,
                'url': link,
               'datetime': date_time
            })

In [3]:
import sqlite3

def get_page_data(id, url):
    conn = sqlite3.connect('iranjibnew.db')
    cursor = conn.cursor()
    creat_table_statement = '''CREATE TABLE IF NOT EXISTS headline(
                                id,
                                title,
                                url,
                                summery,
                                content
                            )'''
    cursor.execute(creat_table_statement)
    r = requests.get(url)
    page_soup = BeautifulSoup(r.content, 'html.parser')
    title = page_soup.find('h1').get_text()
    summery = page_soup.find('div', {'class': 'newssummary'}).get_text()
    news_wrapper = page_soup.find('div', id ='news_wrapper')
    prgs = news_wrapper.find_all('p')
    text = ''
    for p in prgs:
        text += p.get_text() + '\n'

    insert_into_statement = '''INSERT INTO headline VALUES (?,?,?,?,?)'''
    cursor.execute(insert_into_statement, (id,title, url, summery, text))
    conn.commit()
    conn.close()

for id, item in enumerate(data):
    if 'طلا' in item['title'] and 'شهریور' in item['datetime'] :

        get_page_data(id, item['url'])
        
